# Using PH_EXE class

This notebook explains how to use the **PH\_EXE** class inside the **vqe\_step** module. 

This class, given an ansatz and its parent Hamiltonian, provided as a string  Pauli decomposition, computes the associated ground state energy (that should be near zero).

In [ ]:
import logging

In [ ]:
logging.basicConfig(
    format='%(asctime)s-%(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO
    #level=logging.DEBUG
)


In [ ]:
logger = logging.getLogger('__name__')

In [ ]:
import os
import sys
sys.path.append("../")
import numpy as np

In [ ]:
sys.path.append("../../")
# myQLM qpus
from get_qpu import get_qpu
# myQLM qpus
qpu_c = get_qpu("c")
qpu_p = get_qpu("python")

Before doing the siumulation we need to perform several steps:

1. Ansatz Definition and Solving (doing with functions from **ansatzes** module).
2. Parent Hamiltonian Computation (doing with functions from **parent\_hamiltonian** module)

## 1. Ansatz Definition and Solving

We neeed to define an ansatz circuit and solving it for getting its state (see **02_Ansatzes.ipynb**. We can use the functions from **ansatzes** module

### Ansatz definition

First we define an ansatz circuit

In [ ]:
from ansatzes import ansatz_selector, angles_ansatz01

In [ ]:
# Ansatz Configuration
ansatz = "simple01"
nqubits = 6
depth = 2
ansatz_conf = {
    'nqubits' : nqubits,
    'depth' : depth
}
filename = "ansatz_{}_dept_{}_nqubits_{}".format(ansatz, ansatz_conf["depth"], ansatz_conf["nqubits"])
# Create Ansatz Circuit
circuit = ansatz_selector(ansatz, **ansatz_conf)

In [ ]:
%qatdisplay circuit --svg

In [ ]:
# Fixing the parameters of the circuit
circuit, parameters_pdf = angles_ansatz01(circuit)

In [ ]:
%qatdisplay circuit --svg

In [ ]:
parameters_pdf

### Ansatz Solution

Now we need to fix the parameters of the ansatz and solving it(see notebook **02_Ansatzes.ipynb**)

In [ ]:
from ansatzes import SolveCircuit

In [ ]:
folder = "Saving/"

In [ ]:
# Solving  Ansatz Configuration

solve_conf = {
    "nqubits":nqubits,
    "qpu" : qpu_c,
    "parameters" : parameters_pdf,
    "filename": folder + filename,
    "save": True        
}
# Solving Ansatz
solv_ansatz = SolveCircuit(circuit, **solve_conf)
solv_ansatz.run()

In [ ]:
solv_ansatz.state

## 2. Parent Hamiltonian Computation

With the obtained state of the ansatz, we need to compute the Parent Hamiltonian and get its Pauli string decomposition. Here we use the **parent\_hamiltonian** module (see notebook **03_Using_PH_Class.ipynb** for more information)

In [ ]:
from parent_hamiltonian import PH

In [ ]:
# Create PH
# Get teh amplitudes
amplitudes = list(solv_ansatz.state["Amplitude"])
ph_conf = {
    "filename": folder + filename,
    "save": True          
}
ph_object = PH(amplitudes, t_invariant=True, **ph_conf)
ph_object.local_ph()

In [ ]:
ph_object.pauli_pdf

## 4. Computing PH ground state

Now we have all mandatory inputs so we can use **PH_EXE** class for computing the ground state energy of the parent hamiltonian of our input ansatz.
We need to provide for initializing the class:

* *ansatz*: myqlm circuit of the ansatz where the parameters are fixed.
* *pauli_ph*: pandas DataFrame with the Pauli decomposition of the Parent Hamiltonian
* *nqubits*: number of qubits of the ansatz

Additionally, other parameters can be provided as keyword arguments (*kwargs*):
* *qpu*: myqlm QPU unit for simulating the ground state energy
* *nb_shots*: number of shots for measuring GSE.
* *t_inv*: if True for indicating that the Pauli decomposition was done using translational invariance.
* *truncation*: for truncating the number of Pauli strings. It is a positive number and all the Pauli coefficients where its absolute value is lower than $10^{-\text{truncation}}$ will be deleted

The **run** method should be used for the computation.

**BE AWARE**

If the local **PH** was computed using the translational invariance then the obtained Pauli decomposition was obtained for the first qubit only!! For executing the complete step we need to replicate the Pauli strings for all the qubits. This is done automatically by the class **BUT it is mandatory to indicate it by the t_inv key**

In [ ]:
from vqe_step import PH_EXE

In [ ]:
vqe_conf = {
    "t_inv":True,
    "qpu" : qpu_c,
    "nb_shots": 0,
    "truncation": None,
    "filename": folder + filename,
    "save": True        
}

ansatz_circuit = solv_ansatz.circuit
pauli_ph = ph_object.pauli_pdf
nqubits = ansatz_conf["nqubits"]
exe_ph = PH_EXE(ansatz_circuit, pauli_ph, nqubits, **vqe_conf)

In [ ]:
exe_ph.run()

The results will be stored in the attribute **pdf_result** (that will be a pandas DataFrame)

In [ ]:
exe_ph.pdf_result

In [ ]:
exe_ph.pdf_info

In [ ]:
exe_ph.pdf

If save was required the complete information (axis 1 concatenation of *exe_ph.pdf_result* and *exe_ph.pdf_result*) will be saved as: folder + filename+*_phexe.csv*

## 5. run_ph_execution

Inside the module **vqe_step** the **run_ph_execution** function can be found. This function uses typical **CSV** files from **ansatzes** and **parent_hamiltonian** modules for executing a **VQE** step. The input of this function is a **kwargs** (called **configuration**) where following keywords are processed:

* base_fn: Base name for the files generated by **ansatzes** and **parent_hamiltonian**. The pattern of the name can be: 
    * **ansatz\_{}\_nqubits\_{}\_depth\_{}\_qpu_ansatz\_{}**. From this kind of filenames the code extracts directly:
        * ansatz
        * nqubits
        * depth
        * The following files must exist:
            * **ansatz\_{}\_nqubits\_{}\_depth\_{}\_qpu_ansatz\_{}\_parameter.csv**: with the angles of the ansatz
            * **ansatz\_{}\_nqubits\_{}\_depth\_{}\_qpu_ansatz\_{}\_pauli.csv**: with the Pauli decomposition of the **PH**.
    * **nqubits\_{}\_depth\_{}\_qpu\_{}**. From this kind of filenames the code extracts directly:
        *  ansatz: the ansatz will be fixed by defalut to *simple01*
        *  nqubits
        *  depth
        * The following files must exist:
            * **nqubits\_{}\_depth\_{}\_qpu_ansatz\_{}\_parameter.csv**: with the angles of the ansatz
            * **nqubits\_{}\_depth\_{}\_qpu_ansatz\_{}\_pauli.csv**: with the Pauli decomposition of the **PH**.
    * Additionally filenames without the **\
    * _qpu_ansatz** can be given as valid filenames. Corresponding **_parameter.csv** and **_pauli.csv** files must exist
* qpu_ph: name with the qpu for executing the **VQE** step simulation.
* nb_shots: for setting the  number of shots for the **VQE** step.
* t_inv: True if the ansatz is transaltional invariant.
* truncation: integer for truncating the Pauli coefficients
* save: for saving the results. The results will be saved with the following pattern that depends on the *base_fn*:
    * **ansatz\_{}\_nqubits\_{}\_depth\_{}\_qpu_ansatz\_{}\_ph\_exe.csv**
    * **nqubits\_{}\_depth\_{}\_qpu_ansatz\_{}\_ph\_exe.csv**
    * If in the base_fn the name does not have the **\_qpu_ansatz** the created file does not have too.

## 6. Command Line execution

The **run_ph_execution** function from module **vqe_step**  can be executed from command line. Several arguments can be passed for configuring the **VQE** step computation. A help can be obtained by:

**python vqe_step.py -h**

Arguments can be:

* -basefn BASE_FN: Base Filename for Loading Files
* -nb_shots NB_SHOTS    Number of shots
* -truncation TRUNCATION: Truncation for Pauli coeficients.
* -qpu_ph QPU_PH: QPU for **VQE** simulation: [qlmass, python, c]
* --t_inv: Setting translational invariant of the ansatz
* --save: For storing results

**Example**

We are going to use the example provided in the **02_Ansatzes.ipynb** and **03_Using_PH_Class.ipynb** notebooks where we have used the **ansatzes.py** module for doing computations for a **simple_02** ansatz of 6 **qubits** and **depth** 2, and the **parent_hamiltonian.py** module for computing the corresponding Pauli decomposition of its parent Hamiltonian. We need to have the following files in the folder **Saves**:

* ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_c_parameters.csv
* ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_c_state.csv
* ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_c_solve_ansatz_time.csv
* ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_c_ph_time.csv
* ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_c_pauli.csv

Now we can use the **vqe_step.py** module for executing a **VQE** step by using the following command (we have set truncation to 3 so all Pauli coefficients lower than $10^{-3}$ will be pruned):

    python vqe_step.py -basefn Saves/ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_c -qpu_ph c -truncation 3 --save

Now in the **Saves** folder, we should find the following fils (in addition to the other ones)

* ansatz_simple02_nqubits_6_depth_2_qpu_ansatz_c_phexe.csv

## 7. Massive VQE step computations

As for the case of masive ansatz state and parent hamiltonian computations (see Notebook **02_Ansatzes.ipynb and 03_Using_PH_Class.ipynb**) we can execute masive **VQE** step computations.For this the 2 following files can be used:

* **vqe\_step.json**: JSON file with the configuration for the **VQE** step computations. For each desired computation a complete dictionary should be provided. The keys are:
    * save: for saving or not the results
    * t_inv: for setting if the ansatz is, or not, translational invariant.
    * base_fn: base name for the inputs files.
    * nb_shots: number of shots for **VQE** step.
    * qpu_ph: qpu for executing **VQE** step.
    * truncation: for tuncating Pauli coefficients.
* **launch_vqe_step.py**: This scripts procces the before **JSON** file creating a complete list of all posible **VQE** calculations. By providing different arguments a selected configuration (or all of them) can be chosen. For getting help use: **launch_vqe_step.py -h**. Following arguments can be provided:
  * **--count**: Getting the number of posible **VQE** steps from the **vqe\_stepjson** JSON file.
  * **--all**: for selecting all the posible **VQE** steps from the **vqe\_stepjson** JSON file.
  * **-id ID**: for selecting a single (the **ID** one) **VQE** step from the **vqe\_stepjson** JSON file.
  * **--print**: for printing the **VQE** step configuration.
  * **--exe**: for executing the **VQE** execution indicated by **--all** or by **-id ID**.